### There are various statistical hypothesis tests we can implement to estimate the average treatment effect depending on the data type and distributional properties.

There are three major data types: 

    - Continuous Data
    - Count Data
    - Proportional/Categorical Data

Each dataset has a different type of distributional property which resembles the data generating process for a random variable. These distributional properties determine the shape of the distribution such as the skewness and the dispersion of the data.

    - Normally distributed. This distribution requires a mean and standard deviation parameter to characterize the entire distribution.
    - Poisson or negative binomial distribution (for count data). Poisson distribution shows the number of events (or successes) within a certain time. This distribution is skewed towards zero at low sample sizes, but converges to normal distribution as the λ parameter converges to infinity.
    - Bernoulli distribution (for proportional data). Probability of 0 or 1 outcome is the sole parameter to estimate.
    - Zero-inflated: Truncated distribution such as two-part distributions, one for the proportion of zeros, and second for the non-zero portion. These datasets typically have too many observations with zero values.

In [3]:
# load packages
import numpy as np
from scipy import stats
from statsmodels.stats.proportion import proportions_ztest
import random

### T-Test and Mann-Whitney U Test (Wilcoxon Ranked-Sum Test)

T-test checks whether the sample means are equal or not. H0:μ1=μ2 vs H1:μ1≠μ2 . T-test is preferable if your dataset resembles a normal distribution.

MWU checks whether two randomly selected numbers X and Y come from the same distribution, or their medians are equal, or alternatively the following: H0:prob(X≥Y)=prob(X≤Y) vs H1:prob(X≥Y)=prob(X≤Y) . MWU test is advised if your data are not normally distributed.

We will show the simple python implementation as follow. We create two random variables x1 and x2 and implement a T-test and a MWU test. 
When p-value < 0.05, we can safely reject H0,
otherwise,  H0 is likely to be true.

In [4]:
np.random.seed(1234)
mu1, mu2 = 0.5, 0.55
sig1, sig2 = 1.4, 2
x1 = sig1 * np.random.randn(100) + mu1
x2 = sig2 * np.random.randn(100) + mu2
 
def t_test(x1, x2):
    """ Return  T-statistic and p-value"""
    # test statistic
    return stats.ttest_ind(x1, x2, equal_var=False)
 
def mwu_test(x1, x2):
    return stats.mannwhitneyu(x1, x2, use_continuity=True,
                              alternative='two-sided')
 
print(t_test(x1,x2))

print(mwu_test(x1,x2))

Ttest_indResult(statistic=0.4118821010902959, pvalue=0.6809189471796941)
MannwhitneyuResult(statistic=5175.0, pvalue=0.6698372665970904)


### Proportion or Chi-Square Test:
When you observe data as failure (0) and success (1), you can implement a proportions z-test or a chi-square test. Suppose you have two datasets, control group is represented by x1 = [0, 0, 1, 1, 1, 0] where 0 represents failure for an event, and 1 represents success for an event. There are 6 events (page views), and 3 successes (page-views). 
And let's suppose the treatment group is represented by the vector x2 = [0, 1, 1, 0, 1, 1, 1] where there are 7 events and 4 successes. We can implement a proportions or chi-square test to determine whether the samples are different than each other. We observe that the average success rate is 0.5 for group 1, and 4/7 for group 2. We can then compare the two to decide if the treatment group have statistically higher success rates. 

Below we implement the same test with randomly generated data.

In [6]:
np.random.seed(1234)
p1, p2 = 0.10, 0.15
x1 = np.random.binomial(1, p1, size=100)
x2 = np.random.binomial(1, p2, size=100)
 
def proportion_test(x1, x2, chi_sq=True, chi_sq_correction=False):
    """This function performs a proportion (z-test or chi-sq) on
    binary-categorical data that takes 0 or 1 values
    Inputs:
        x1: list or numpy array
        x2: list or numpy array
        chi_sq = True for chi squared test, and False for proportions z-test.
 
    The function creates the necessary inputs for the proportions z-test or the
    chi-square test functions in python. For the proportions z-test, the
    function calculates the success counts (count array) and event counts
    (n_obs array).
 
    For the chi-square test, the function constructs the contingency table
    which shows the total number of successes and failures by each group.
 
    Output: Test statistics and relevant metrics.
    """
 
    if type(x1) is list:
        x1 = np.array(x1)
 
    if type(x2) is list:
        x2 = np.array(x2)
 
    n1, n2 = len(x1), len(x2)
    m1, m2 = len(x1[x1.astype(bool)]), len(x2[x2.astype(bool)])
    p1, p2 = m1/n1, m2/n2
    ate = p2 - p1
    pct_uplift = ate/p1
 
    if chi_sq:
        cont_table = [[m1, n1-m1], [m2, n2-m2]]
        chisqstat, pval, dof = \
            stats.chi2_contingency(cont_table,
                                   correction=chi_sq_correction)[:3]
        sig_chisq = "Significant" if pval < 0.05 else "Not Significant"
        return cont_table, 'chi_sq', ate, pct_uplift, \
        chisqstat, pval, dof, sig_chisq
 
    else:
        count = np.array([m1, m2])
        n_obs = np.array([n1, n2])
        zstat, pval = proportions_ztest(count, n_obs)
 
        sig_ztest_prop = 'Significant' if pval < 0.05 else "Not Significant"
        return [count, n_obs], 'z_test_prop', ate, \
        pct_uplift, zstat, pval, sig_ztest_prop
 
 
count_nobs,test,ate,pct_uplift,chisqstat,pval,dof,sig_chisq = \
    proportion_test(x1, x2, chi_sq=True)
count_nobs1,test1,ate1,pct_uplift1,zstat,pval1,sig_ztest_prop = \
    proportion_test(x1, x2, chi_sq=False)
 
print('Chisquared test p-value = {}'.format(pval))
 
print('Z test p-value = {}'.format(pval))

Chisquared test p-value = 0.07069592991793984
Z test p-value = 0.07069592991793984


### Regression Tests for Count Data:

When you work with count data with integer values (e.g., number of times you log in to the application), the distribution from which this random variable is drawn is likely to be skewed. In other words, you may not assume that your variable is normally distributed and you may not use a T-test when the sample size is relatively small. While you can implement a non-parametric test such as the MWU test, regression based tests that use count-data distributions such as Poisson or Negative binomial distribution are advised for count data. There are a few alternatives when you implement a regression based tests for count data: Poisson, Gamma, Negative Binomial, Hurdle Regression, Zero-inflated Poisson Regression, and Zero-inflated Negative Binomial Regression.

Here is an example in Python to estimate if the treatment and control group means are different from each other:

In [7]:
from statsmodels.discrete.count_model import ZeroInflatedPoisson
from statsmodels.discrete.count_model import ZeroInflatedNegativeBinomialP
import statsmodels.api as sm
 
def discrete_model_regression(x1, x2, model='Poisson'):
    """This function fits a discrete count model (such as Poisson or
    negative binomial) using the treatment group as the sole
    explanatory variable.
    Choose model: [Poisson, NegativeBinomial,
                   ZeroInflatedPoisson, ZeroInflatedNegativeBinomial] """
 
    # prepare constant & treatment dummy vector
    x1 = np.hstack((x1, [1] * len(x1), [0]*len(x1))).reshape(3, len(x1)).T
    x2 = np.hstack((x2, [1] * len(x1), [1]*len(x2))).reshape(3, len(x2)).T
    x = np.concatenate((x1, x2))
    y = x[:, 0]
    X = x[:, 1:]
 
    if model == 'Poisson':
        reg = sm.GLM(y, X, family=sm.families.Poisson()).fit(disp=0)
    elif model == 'NegativeBinomial':
        reg = sm.GLM(y, X, family=sm.families.NegativeBinomial()).fit(disp=0)
    else:
        reg = model(y, X).fit(disp=0)
 
    ate = reg.params[1]
    tstat = reg.tvalues[1]
    pval = reg.pvalues[1]
 
    return ate, tstat, pval
 
np.random.seed(1234)
mu1, mu2 = 10, 11
x1 = np.random.poisson(lam=mu1, size=1000)
x2 = np.random.poisson(lam=mu2, size=1000)
 
print(discrete_model_regression(x1, x2, model='Poisson'))
 
print(discrete_model_regression(x1, x2, model='NegativeBinomial'))
 
print(t_test(x1,x2))

(0.09209280321031127, 6.647413412641464, 2.9828835176825425e-11)
(0.09209280321031389, 1.967035588846957, 0.049179108010313395)
Ttest_indResult(statistic=-6.565112235299977, pvalue=6.612354352350283e-11)


C:\Users\bshakeri\AppData\Local\anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


### Two-Part Test for Zero-Inflated Data

When we have a dataset that have too many zeros, we can implement a two-part test procedure, where the test statistic is formed by using two sets of information from the data: (1) The fraction of non-zero to all observations; and (2) the value of non-zero observations.

This test combines a chi-square test for part 1; and either a t-test or Mann-Whitney U test for the second part. The test is appropriate for continuous data. We can also implement this test to count data.


In [8]:
# Two-part hypothesis test for zero-inflated datasets. The test is
# (count or continuous)
# Call the 'two_part_test' function using x1 and x2 as the input arrays.
def two_part_test(x1, x2, firstpart='mwu'):
    """ Compute the test statistic of the two part test that combines the
    Z-proportion chisquare test and the wilcoxon ranked (mwu) test.
    The final test statistic is Z2 + W2, which is distributed
    with a chisq distribution with 2 degrees of freedom.
 
    Return: Test Statistic, p-value
    """
 
    def part1_chi2(n1, n2, m1, m2):
        """Z-proportion test (similar to stats.scipy.chi2_contigency function)
        Return Chi-Sq Test Statistic From count data of arrays x1 and x2
        Returns: Chi-Sq Test Statistic, and the P-Value"""
 
        p1, p2 = m1/n1, m2/n2
        phat = (m1+m2) / (n1+n2)
        qhat = 1 - phat
 
        Z = (np.abs(p1 - p2) - (1/(2*n1) + 1/(2*n2))) \
            / np.sqrt(phat * qhat * (1/n1 + 1/n2))
        return Z**2, 1 - stats.chi2.cdf(Z**2, 1)
 
    def part2_wilcoxon_ranked(x1_nonzero, x2_nonzero, m1, m2):
        """Return Square of standardized Wilcoxon-Ranked (mwu) test statistic
        and its p-value using only the non-zero count data.
 
        Note that the standard deviation of the statistic is calculated using
        a normal approximation without tie correction for simplicity."""
 
        # test statistic
        U = stats.mannwhitneyu(x1_nonzero, x2_nonzero)[0]
        # normality approximation
        mu_U = m1 * m2 / 2
        sig_U = np.sqrt(m1 * m2 * (m1 + m2 + 1) / 12)
 
        W = (np.abs(U-mu_U) - 1/2)/sig_U
        return W**2, 1 - stats.chi2.cdf(W**2, 1)
 
    def part2_ttest(x1_nonzero, x2_nonzero):
        """ Return Square of T-statistic on the non-zero population"""
        # test statistic
        T = stats.ttest_ind(x1_nonzero, x2_nonzero, equal_var=False)[0]
 
        return T**2, 1 - stats.chi2.cdf(T**2, 1)
 
    if type(x1) is list:
        x1 = np.array(x1)
 
    if type(x2) is list:
        x2 = np.array(x2)
 
    x1_nonzero = x1[x1 != 0]
    x2_nonzero = x2[x2 != 0]
 
    n1, n2 = len(x1), len(x2)
    m1, m2 = len(x1_nonzero), len(x2_nonzero)
    p1, p2 = m1/n1, m2/n2
    m1, n1, p1, m2, n2, p2
 
    print(m1, n1, p1, m2, n2, p2)
    Z2 = part1_chi2(n1, n2, m1, m2)[0]
 
    if firstpart == 'ttest':
        W2 = part2_ttest(x1_nonzero, x2_nonzero)[0]
    elif firstpart == 'mwu':
        W2 = part2_wilcoxon_ranked(x1_nonzero, x2_nonzero, m1, m2)[0]
    ChiSq = Z2 + W2
 
    return ChiSq, 1 - stats.chi2.cdf(ChiSq, 2)
 
# suppose we have 1000 observations in x1 and x2, and we randomize the
# instance of non-zero observations, which will be at most 100.
# and then pick a random distribution for the non-zero portion
random.seed(1234)
np.random.seed(1234)
n_nonzero1 = random.randint(0, 100)
n_nonzero2 = random.randint(0, 100)
zeros1 = np.array([0] * (1000 - n_nonzero1))
zeros2 = np.array([0] * (1000 - n_nonzero2))
mu1, mu2 = 0.5, 0.55
sig1, sig2 = 1.4, 2
x1_nonzero = sig1 * np.random.randn(n_nonzero1) + mu1
x2_nonzero = sig2 * np.random.randn(n_nonzero2) + mu2
x1 = np.concatenate((zeros1, x1_nonzero))
x2 = np.concatenate((zeros2, x2_nonzero))
 
print(two_part_test(x1, x2, firstpart='ttest'))
print(two_part_test(x1, x2, firstpart='mwu'))

99 1000 0.099 56 1000 0.056
(13.074172531888697, 0.001448703492055503)
99 1000 0.099 56 1000 0.056
(13.444055548328759, 0.0012040941058265586)
